<a href="https://colab.research.google.com/github/GenaroHacker/creating_chord_collection/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Set Up
%%capture
!git clone https://github.com/GenaroHacker/creating_chord_collection.git
from creating_chord_collection.transposable_figures import transposable_figures

In [24]:
# @title Chord
class GuitarChord:
    all_notes = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
    open_string_notes = ["E", "B", "G", "D", "A", "E"]

    def __init__(self, root, chord_type, transposable_figures, *, starting_fret=0, finger_ascending):
        self.root = root
        self.chord_type = chord_type
        self.starting_fret = starting_fret
        self.finger_ascending = finger_ascending
        self.transposable_figures = transposable_figures

    def __str__(self):
        return f"({repr(self.root)}, {repr(self.chord_type)}, finger_ascending={self.finger_ascending}, starting_fret={self.starting_fret})"

    def get_notes(self, include_octaves=False, include_frequencies=False):
        notes_with_octaves = {}
        notes_with_frequencies = {}
        base_octaves = [4, 3, 3, 3, 2, 2]
        open_string_frequencies = {
            "E2": 82.41,  # 6th string
            "A2": 110,    # 5th string
            "D3": 146.83, # 4th string
            "G3": 196,    # 3rd string
            "B3": 246.94, # 2nd string
            "E4": 329.63  # 1st string
        }

        for string_number, (open_note, finger_position) in enumerate(zip(GuitarChord.open_string_notes, self.finger_ascending), start=1):
            if finger_position is None:
                # Skip strings that are not being played
                continue

            # Determine the correct frequency key for the open string
            base_octave = base_octaves[string_number - 1]
            freq_key = f"{open_note}{base_octave}"

            if finger_position == 0:
                # Handle open string
                note = open_note
                octave = base_octave
                frequency = open_string_frequencies[freq_key]
            else:
                # Calculate the note considering the starting fret
                note_index = (GuitarChord.all_notes.index(open_note) + finger_position + self.starting_fret) % len(GuitarChord.all_notes)
                note = GuitarChord.all_notes[note_index]

                # Calculate the octave
                octave_increase_from_fret = (finger_position + self.starting_fret) // 12
                octave = base_octave + octave_increase_from_fret

                # Increment the octave if we cross a B to C transition
                if "B" in GuitarChord.all_notes[GuitarChord.all_notes.index(open_note):] and note == "C":
                    octave += 1

                # Calculate frequency if needed
                if include_frequencies:
                    base_frequency = open_string_frequencies[freq_key]
                    semitones_from_open = finger_position + self.starting_fret
                    frequency = base_frequency * (2 ** (semitones_from_open / 12))

            if include_octaves:
                notes_with_octaves[string_number] = (note, octave)
            if include_frequencies:
                notes_with_frequencies[string_number] = (note, octave, frequency)

        if include_frequencies:
            return notes_with_frequencies
        if include_octaves:
            return notes_with_octaves
        return [note for note in notes_with_octaves.values()]


    def is_open(self):
        return 0 in self.finger_ascending

    def get_absolute_notes(self):
        string_notes = self.get_notes()
        filtered_notes = filter(None, string_notes)
        unique_notes = list(dict.fromkeys(filtered_notes))

        # Create a list starting from the root note
        root_index = GuitarChord.all_notes.index(self.root)
        sorted_notes_order = GuitarChord.all_notes[root_index:] + GuitarChord.all_notes[:root_index]

        # Sort the notes as per their order in sorted_notes_order
        sorted_notes = sorted(unique_notes, key=lambda x: sorted_notes_order.index(x))
        return sorted_notes


    def transpose(self, distance):
        # Helper function to transpose figure
        def transpose_figure(lst, num):
            return [item + num if item is not None else None for item in lst]

        # Helper function to raise specific errors after reverting changes
        def raise_transpose_error(error_type):
            self.root = original_root
            self.finger_ascending = original_finger_ascending
            self.starting_fret = original_starting_fret

            error_messages = {
                "below_0": "Chord transposition results in a note below the 0th fret.",
                "above_12": "Chord transposition results in a note above the 12th fret.",
                "not_equivalent_transposable_figure": "Chord figure is not equivalent to any figure in transposable_figures."
            }
            raise ValueError(error_messages[error_type])

        # Save original state for possible reversion
        original_root = self.root
        original_finger_ascending = self.finger_ascending.copy()
        original_starting_fret = self.starting_fret

        # Return if distance is zero
        if distance == 0:
            return

        # Update root note
        new_note_index = (GuitarChord.all_notes.index(self.root) + distance) % len(GuitarChord.all_notes)
        self.root = GuitarChord.all_notes[new_note_index]

        # Transpose open chords
        if self.is_open():
            if distance < 0:
                raise_transpose_error("below_0")
            elif distance > 0:
                self.finger_ascending = transpose_figure(self.finger_ascending, 1)
                self.starting_fret = max(0, self.starting_fret + distance - 1)
        else:  # Transpose barre chords
            if distance < 0:
                new_starting_fret = self.starting_fret + distance
                if new_starting_fret < 0:
                    raise_transpose_error("below_0")
                elif new_starting_fret == 0:
                    self.finger_ascending = transpose_figure(self.finger_ascending, -1)
                    self.starting_fret = 1  # Keeping the fret at 1
                else:
                    self.starting_fret = new_starting_fret
            else:  # Transpose barre chord to the right
                self.starting_fret += distance

        # Check for errors in transposition
        if any(fret < 0 for fret in self.finger_ascending if fret is not None):
            raise_transpose_error("below_0")
        if self.starting_fret > 9:
            raise_transpose_error("above_12")

        # Check transposability
        transposed_figure = self.finger_ascending if self.starting_fret == 0 else transpose_figure(self.finger_ascending, 1)
        if transposed_figure not in self.transposable_figures:
            raise_transpose_error("not_equivalent_transposable_figure")



# Example
chord = GuitarChord('E', 'm', transposable_figures, finger_ascending=[0, 0, 0, 2, 2, 0], starting_fret=1)
print(chord)
print("String Notes with Octaves:", chord.get_notes(include_octaves=True, include_frequencies=True))
print("Is Open Chord:", chord.is_open())
print("Absolute Notes:", chord.get_absolute_notes())
chord.transpose(1)
print(chord)

('E', 'm', finger_ascending=[0, 0, 0, 2, 2, 0], starting_fret=1)
String Notes with Octaves: {1: ('E', 4, 329.63), 2: ('B', 3, 246.94), 3: ('G', 3, 196), 4: ('F', 3, 174.61128069584953), 5: ('C', 3, 130.8127826502993), 6: ('E', 2, 82.41)}
Is Open Chord: True
Absolute Notes: []
('F', 'm', finger_ascending=[1, 1, 1, 3, 3, 1], starting_fret=1)


In [27]:
# Example
chord = GuitarChord('E', 'm', transposable_figures, finger_ascending=[1, 1, 1, 1, 0, None], starting_fret=11)
print(chord)
print("String Notes with Octaves:", chord.get_notes(include_octaves=True, include_frequencies=True))
print("Is Open Chord:", chord.is_open())
print("Absolute Notes:", chord.get_absolute_notes())


('E', 'm', finger_ascending=[1, 1, 1, 1, 0, None], starting_fret=11)
String Notes with Octaves: {1: ('E', 5, 659.26), 2: ('B', 4, 493.88), 3: ('G', 4, 392.0), 4: ('D', 4, 293.66), 5: ('A', 2, 110)}
Is Open Chord: True
Absolute Notes: []


In [15]:
# @title Collection
import sqlite3
from collections import defaultdict

class ChordCollection:
    def __init__(self):
        self.chords = []

    def load(self, db_path):
        self.chords.clear()
        connection = sqlite3.connect(db_path)
        cursor = connection.cursor()

        cursor.execute('SELECT ROOT, TYPE, STARTING_FRET, STRING_1, STRING_2, STRING_3, STRING_4, STRING_5, STRING_6 FROM TABLE_CHORDS')
        rows = cursor.fetchall()

        for row in rows:
            root, chord_type, starting_fret, *fingers = row
            chord = GuitarChord(root, chord_type, transposable_figures, finger_ascending=fingers, starting_fret=starting_fret)
            self.chords.append(chord)

        connection.close()

    def save(self, db_name):
        connection = sqlite3.connect(db_name)
        cursor = connection.cursor()

        # Create the table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS TABLE_CHORDS (
                ID INTEGER PRIMARY KEY,
                ROOT TEXT,
                TYPE TEXT,
                STARTING_FRET INTEGER,
                STRING_1 INTEGER,
                STRING_2 INTEGER,
                STRING_3 INTEGER,
                STRING_4 INTEGER,
                STRING_5 INTEGER,
                STRING_6 INTEGER
            )
        ''')

        # Insert the chords
        for chord in self.chords:
            cursor.execute('''
                INSERT INTO TABLE_CHORDS (ROOT, TYPE, STARTING_FRET, STRING_1, STRING_2, STRING_3, STRING_4, STRING_5, STRING_6)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (chord.root, chord.chord_type, chord.starting_fret, *chord.finger_ascending))

        connection.commit()
        connection.close()

    def chord_exists(self, new_chord):
        for chord in self.chords:
            if chord.root == new_chord.root and chord.chord_type == new_chord.chord_type and chord.starting_fret == new_chord.starting_fret and chord.finger_ascending == new_chord.finger_ascending:
                return True
        return False

    def extend_barre_chords(self):
        original_chords = self.chords.copy()
        for chord in original_chords:
            counter = 1
            while True:
                try:
                    new_chord = GuitarChord(chord.root, chord.chord_type, chord.transposable_figures, finger_ascending=chord.finger_ascending.copy(), starting_fret=chord.starting_fret)
                    new_chord.transpose(counter)
                    if not self.chord_exists(new_chord):
                        self.chords.append(new_chord)
                    counter += 1
                except ValueError:
                    break


# Example usage
chord_collection = ChordCollection()
chord_collection.load('/content/creating_chord_collection/chord_collection.db')

chord = chord_collection.chords[6]
print(chord)
chord.transpose(1)
print(chord)
chord_collection.save('new_chord_collection.db')
print(f"Collection Size: {len(chord_collection.chords)}")
chord_collection.extend_barre_chords()
print(f"Collection Size: {len(chord_collection.chords)}")




('A', 'm', finger_ascending=[0, 1, 2, 2, 0, None], starting_fret=1)
('A#', 'm', finger_ascending=[1, 2, 3, 3, 1, None], starting_fret=1)
Collection Size: 132
Collection Size: 328
